In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
import string

In [7]:
data = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json",lines=True)

In [8]:
# remove duplicate headlines
data=data.drop(data[data['headline'].duplicated()].index,axis=0)
sarc_cnt = len(data.query('is_sarcastic==1'))
non_sarc_cnt = len(data.query('is_sarcastic==0'))

# Summary of sarcastic lines
print(f'There are {sarc_cnt} sarcastic headlines and {non_sarc_cnt} non-sarcastic headlines')

There are 13552 sarcastic headlines and 14951 non-sarcastic headlines


## Part 2: Data Processing/Cleaning

In [ ]:
# import stopwords from nltk
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line

In [ ]:
data['cleaned'] = data['headline'].apply(clean_headlines)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(2,2))

In [ ]:
labels = data.is_sarcastic

In [ ]:
res = cv.fit_transform(data.iloc[0:2].cleaned)
res = cv.fit_transform(data.cleaned)
res.shape

(28503, 148333)

In [ ]:
df = pd.DataFrame(res.toarray(),columns=cv.get_feature_names_out())

In [ ]:
df

,00 probabl,000 000,000 100,000 115,000 15,000 acr,000 actual,000 adopt,000 airlin,000 american,...,zuckerberg regret,zuckerberg sell,zuckerberg senat,zuckerberg tout,zuckerberg trump,zuckerberg watch,zuckerberg wish,zuckerberg you,zz top,ünite stäte
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=0)

In [ ]:
 test_split = train_test_split(df,labels, test_size=.3)

In [ ]:
feature_train, feature_test, labels_train, labels_test= test_split

In [ ]:
model.fit(feature_train,labels_train)

LogisticRegression(random_state=0, solver='liblinear')

In [ ]:
model.predict_proba(feature_train)

array([[0.81269162, 0.18730838],
       [0.21131565, 0.78868435],
       [0.83364155, 0.16635845],
       ...,
       [0.73603691, 0.26396309],
       [0.78623822, 0.21376178],
       [0.71691896, 0.28308104]])

In [ ]:
confusion_matrix(labels_train, model.predict(feature_train))

array([[10454,     5],
       [    8,  9485]])

In [ ]:
confusion_matrix(labels_test, model.predict(feature_test))

array([[4063,  429],
       [2412, 1647]])

In [ ]:
print(classification_report(labels_test, model.predict(feature_test)))

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.64      8551



In [ ]:
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

In [ ]:
model.fit(feature_train,labels_train)

LogisticRegression(C=10.0, random_state=0, solver='liblinear')

In [ ]:
print(classification_report(labels_test, model.predict(feature_test)))

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.65      8551



In [ ]:
print ([i for i in range(2,15,2)])

[2, 4, 6, 8, 10, 12, 14]


In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm (range (2,15,2)):
    if i in[0,1,10]: continue
    model = LogisticRegression(solver='liblinear', C=i, random_state=0)
    model.fit(feature_train,labels_train)
    print("C = ", i)
    print(classification_report(labels_test, model.predict(feature_test)))

  0%|                                                     | 0/7 [00:00<?, ?it/s]

C =  2


 14%|██████▍                                      | 1/7 [00:51<05:08, 51.35s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.65      8551

C =  4


 29%|████████████▊                                | 2/7 [01:40<04:09, 49.94s/it]

...

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.65      8551

C =  12


 86%|██████████████████████████████████████▌      | 6/7 [04:09<00:36, 36.51s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.64      8551

C =  14


100%|█████████████████████████████████████████████| 7/7 [04:58<00:00, 42.67s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.65      0.64      8551
weighted avg       0.71      0.67      0.64      8551



## Part 3: Data Processing/Cleaning 3


In [5]:
import nltk

In [9]:
# import stopwords from nltk
nltk.download('stopwords')
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
data['cleaned'] = data['headline'].apply(clean_headlines)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(3,3))

In [12]:
labels = data.is_sarcastic

In [13]:
res = cv.fit_transform(data.iloc[0:2].cleaned)
res = cv.fit_transform(data.cleaned)
res.shape

(28503, 150155)

In [ ]:
df = pd.DataFrame(res.toarray(),columns=cv.get_feature_names_out())

In [1]:
df

NameError: ignored

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
model = LogisticRegression(solver='liblinear', random_state=0)

In [16]:
test_split = train_test_split(res,labels, test_size=.3)

In [17]:
feature_train, feature_test, labels_train, labels_test= test_split

In [18]:
model.fit(feature_train,labels_train)

LogisticRegression(random_state=0, solver='liblinear')

In [19]:
model.predict_proba(feature_train)

array([[0.81696024, 0.18303976],
       [0.76230978, 0.23769022],
       [0.21536269, 0.78463731],
       ...,
       [0.81696024, 0.18303976],
       [0.78450311, 0.21549689],
       [0.78450311, 0.21549689]])

In [20]:
confusion_matrix(labels_train, model.predict(feature_train))

array([[10411,     0],
       [   50,  9491]])

In [21]:
confusion_matrix(labels_test, model.predict(feature_test))

array([[4488,   52],
       [3744,  267]])

In [22]:
print(classification_report(labels_test, model.predict(feature_test)))

              precision    recall  f1-score   support

           0       0.55      0.99      0.70      4540
           1       0.84      0.07      0.12      4011

    accuracy                           0.56      8551
   macro avg       0.69      0.53      0.41      8551
weighted avg       0.68      0.56      0.43      8551



In [23]:
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

In [24]:
model.fit(feature_train,labels_train)

LogisticRegression(C=10.0, random_state=0, solver='liblinear')

In [25]:
print(classification_report(labels_test, model.predict(feature_test)))

              precision    recall  f1-score   support

           0       0.54      0.99      0.70      4540
           1       0.84      0.06      0.11      4011

    accuracy                           0.55      8551
   macro avg       0.69      0.52      0.41      8551
weighted avg       0.68      0.55      0.42      8551



In [ ]:
print ([i for i in range(2,15,2)])

[2, 4, 6, 8, 10, 12, 14]


In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm (range (2,15,2)):
    if i in[0,1,10]: continue
    model = LogisticRegression(solver='liblinear', C=i, random_state=0)
    model.fit(feature_train,labels_train)
    print("C = ", i)
    print(classification_report(labels_test, model.predict(feature_test)))

  0%|                                                     | 0/7 [00:00<?, ?it/s]

C =  2


 14%|██████▍                                      | 1/7 [00:51<05:08, 51.35s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.65      8551

C =  4


 29%|████████████▊                                | 2/7 [01:40<04:09, 49.94s/it]

...

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.65      8551

C =  12


 86%|██████████████████████████████████████▌      | 6/7 [04:09<00:36, 36.51s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.66      0.64      8551
weighted avg       0.71      0.67      0.64      8551

C =  14


100%|█████████████████████████████████████████████| 7/7 [04:58<00:00, 42.67s/it]

              precision    recall  f1-score   support

           0       0.63      0.90      0.74      4492
           1       0.79      0.41      0.54      4059

    accuracy                           0.67      8551
   macro avg       0.71      0.65      0.64      8551
weighted avg       0.71      0.67      0.64      8551



In [ ]:
## Part 2: Data Processing/Cleaning

# import stopwords from nltk
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line

data['cleaned'] = data['headline'].apply(clean_headlines)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(2,2))

labels = data.is_sarcastic

res = cv.fit_transform(data.iloc[0:2].cleaned)
res = cv.fit_transform(data.cleaned)
res.shape

df = pd.DataFrame(res.toarray(),columns=cv.get_feature_names_out())

df

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression(solver='liblinear', random_state=0)

 test_split = train_test_split(df,labels, test_size=.3)

feature_train, feature_test, labels_train, labels_test= test_split

model.fit(feature_train,labels_train)

model.predict_proba(feature_train)

confusion_matrix(labels_train, model.predict(feature_train))

confusion_matrix(labels_test, model.predict(feature_test))

print(classification_report(labels_test, model.predict(feature_test)))

model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

model.fit(feature_train,labels_train)

print(classification_report(labels_test, model.predict(feature_test)))

print ([i for i in range(2,15,2)])

from tqdm import tqdm

for i in tqdm (range (2,15,2)):
    if i in[0,1,10]: continue
    model = LogisticRegression(solver='liblinear', C=i, random_state=0)
    model.fit(feature_train,labels_train)
    print("C = ", i)
    print(classification_report(labels_test, model.predict(feature_test)))

IndentationError: unexpected indent (1524009363.py, line 37)